In [1]:
import pandas as pd
import numpy as np
import glob
import csv
from collections import defaultdict
import re
import json

In [2]:
election = pd.read_csv("US_County_Level_Presidential_Results_08-16.csv")

In [3]:
election.dropna(axis=1, inplace=True)

## Collecting Census Data

In [4]:
import urllib.request
import ast


class Census:
    def __init__(self, key):
        self.key = key

    def get(self, fields, geo, year=2012, dataset='sf1'):
        fields = [','.join(fields)]
        base_url = 'http://api.census.gov/data/%s/%s?key=%s&get=' % (str(year), dataset, self.key)
        query = fields
        for item in geo:
            query.append(item)
        add_url = '&'.join(query)
        url = base_url + add_url
        print(url)
        req = urllib.request.Request(url)
        response = urllib.request.urlopen(req)
        return response.read()

In [5]:
census = Census('6c3fba83f5ccd2adb2fd3894045431e20e3ec5cb')

In [6]:
fieldsToGet = dict({
    ('ST', 'State FIPS'),
    ('COUNTY', 'County FIPS Code'),
    ('NAME', 'County Name'),
    ('B00002_001E', '# of Housing Units'),
    ('B01001_001E', 'Population'),
    ('B01001_002E', '# Male'),
    ('B01001_026E', '# Female'),
    ('B01001A_001E', '# White'),
    ('B01001A_002E', '# White Male'),
    ('B01001A_017E', '# White Female'),
    ('B01001B_001E', '# Black'),
    ('B01001B_002E', '# Black Male'),
    ('B01001B_017E', '# Black Female'),
    ('B01001H_001E', '# White Not Hispanic'),
    ('B01001H_002E', '# White Not Hispanis Male'),
    ('B01001H_017E', '# White Not Hispanis Female'),
    ('B01001I_001E', '# Hispanic'),
    ('B01001I_002E', '# Hispanic Male'),
    ('B01001I_017E', '# Hispanic Female'),
    ('B01002_001E', 'Median Age'),
    ('B01002_002E', 'Median Age Male'),
    ('B01002_003E', 'Median Age Female'),
    ('B05001_002E', 'Citizen, born in US'),
    ('B05001_005E', 'Citizen by Naturalization'),
    ('B05002_014E', 'Foreign born naturalized'),
    ('B06008_002E', 'Never Married'),
    ('B06008_003E', 'Currently Married'),
    ('B06008_004E', 'Divorced'),
    ('B06008_005E', 'Separated'),
    ('B06009_001E', 'Education Count'),
    ('B06009_002E', 'Less Than HS'),
    ('B06009_003E', 'HS or equal'),
    ('B06009_004E', 'Some College-AA'),
    ('B06009_005E', 'Bachelor'),
    ('B06009_006E', 'Graduate'),
    ('B06009_007E', 'Total Born In State education'),
    ('B06009_008E', 'In State-Less than HS'),
    ('B06009_009E', 'In State-HS or equal'),
    ('B06009_010E', 'In State-Some College'),
    ('B06009_011E', 'In State-Bachelor'),
    ('B06009_012E', 'In State-Graduate'),
    ('B06009_013E', 'Born in Other State Education'),
    ('B06009_014E', 'Other State-Less than HS'),
    ('B06009_015E', 'Other State-HS or equal'),
    ('B06009_016E', 'Other State-Some College'),
    ('B06009_017E', 'Other State-Bachelor'),
    ('B06009_018E', 'Other State-Graduate'),
    ('B06010_001E', 'Total Reporting Income'),
    ('B06010_002E', 'Total Reporing 0 Income'),
    ('B06010_003E', 'Total Reporting with Income'),
})

fieldsToGet2 = dict({
    ('ST', 'State FIPS'),
    ('COUNTY', 'County FIPS Code'),
    ('NAME', 'County Name'),
    ('B06010_004E', 'Income < 10K'),
    ('B06010_005E', 'Income 10-15K'),
    ('B06010_006E', 'Income 15-25K'),
    ('B06010_007E', 'Income 25-35K'),
    ('B06010_008E', 'Income 35-50K'),
    ('B06010_009E', 'Income 50-65K'),
    ('B06010_010E', 'Income 65-75K'),
    ('B06010_011E', 'Income > 75K'),
    ('B06011_001E', 'Median Income'),
    ('B06011_002E', 'Median Income Born In State'),
    ('B06011_003E', 'Median Income Born Other State'),
    ('B06011_004E', 'Median Income Born Outside US, Native'),
    ('B06011_005E', 'Median Income Foreign Born'),
    ('B06012_001E', 'Total In Poverty Pool'),
    ('B06012_002E', '100% below povery line'),
    ('B06012_003E', '1-1.5X of poverty line'),
    ('B06012_004E', 'At above 150% poverty line'),
    ('B06012_006E', 'Born In State below 100% poverty'),
    ('B06012_007E', 'Born In State 1-1.5X of poverty line'),
    ('B06012_008E', 'Born In State at/above 1.5X poverty line'),
    ('B06012_010E', 'Born Other State below 100% poverty'),
    ('B06012_011E', 'Born Other State 1-1.5X of poverty line'),
    ('B06012_012E', 'Born Other State at/above 1.5X poverty line'),
    ('B06012_018E', 'Foreign Born below 100% poverty'),
    ('B06012_019E', 'Foreign Born 1-1.5X of poverty line'),
    ('B06012_020E', 'Foreign Born at/above 1.5X poverty line'),
    ('B07001_001E', 'Total Moving'),
    ('B07001_017E', 'Same house 1 year ago'),
    ('B07001_033E', 'Moved Within County'),
    ('B07001_049E', 'Moved Within Same State'),
    ('B07001_065E', 'Moved From Different State'),
    ('B07001_081E', 'Moved From Abroad'),
    ('B07004A_001E', 'Total Moving White'),
    ('B07004A_002E', 'Same house 1 year ago White'),
    ('B07004A_003E', 'Moved Within County White'),
    ('B07004A_004E', 'Moved Within Same State White'),
    ('B07004A_005E', 'Moved From Different State White'),
    ('B07004A_006E', 'Moved From Abroad White')
    })

fieldsToGet3 = dict({
    ('ST', 'State FIPS'),
    ('COUNTY', 'County FIPS Code'),
    ('NAME', 'County Name'),
    ('B19101_002E', 'Family Income < 10K'),
    ('B19101_003E', 'Family Income 10-15K'),
    ('B19101_004E', 'Family Income 15-20K'),
    ('B19101_004E', 'Family Income 20-25K'),
    ('B19101_006E', 'Family Income 25-30K'),
    ('B19101_007E', 'Family Income 30-35K'),
    ('B19101_008E', 'Family Income 35-40K'),
    ('B19101_009E', 'Family Income 40-45K'),
    ('B19101_010E', 'Family Income 45-50K'),
    ('B19101_011E', 'Family Income 50-60K'),
    ('B19101_012E', 'Family Income 60-75K'),
    ('B19101_013E', 'Family Income 75-100K'),
    ('B19101_014E', 'Family Income 100-125K'),
    ('B19101_015E', 'Family Income 125-150K'),
    ('B19101_016E', 'Family Income 150-200K'),
    ('B19101_017E', 'Family Income > 200K'),
    ('B08006_001E', 'Total Means of Transportation Work'),
    ('B08006_002E', 'Work by car, truck, van'),
    ('B08006_003E', 'Drive Alone'),
    ('B08006_004E', 'Car pooled'),
    ('B08006_008E', 'Public Transportation'),
    ('B08006_012E', 'Rail Road'),
    ('B08006_013E', 'Ferry Boat'),
    ('B08006_014E', 'Bicycle'),
    ('B08006_015E', 'Walked'),
    ('B08006_017E', 'Work at Home'),
    ('B08007_002E', 'Work in State'),
    ('B08007_003E', 'Work in County'),
    ('B08007_004E', 'Work outside County'),
    ('B08007_005E', 'Work outside State'),
    ('B08014_002E', 'No Vehicle Owned'),
    ('B08014_003E', '1 vehicle'),
    ('B08014_004E', '2 vehicles'),
    ('B08014_005E', '3 vehicles'),
    ('B08014_006E', '4 vehicles'),
    ('B08014_007E', '5 or more vehicles'),
    })

# CHANGE ME IF YOU WANT DATA FROM DIFFERENT YEARS ONY 2011 - 2015 though

In [7]:
year = 2012

In [8]:
AllCounty = census.get(list(fieldsToGet.keys()), 
                   ['for=county:*', 'in=state:*'], 
                   year=year, 
                   dataset='acs5')
AllCounty2 = census.get(list(fieldsToGet2.keys()), 
                   ['for=county:*', 'in=state:*'], 
                   year=year, 
                   dataset='acs5')
AllCounty3 = census.get(list(fieldsToGet3.keys()),
                 ['for=county:*', 'in=state:*'], 
                   year=year, 
                   dataset='acs5')

http://api.census.gov/data/2012/acs5?key=6c3fba83f5ccd2adb2fd3894045431e20e3ec5cb&get=B06010_003E,B05002_014E,B06009_013E,B01001H_017E,B06009_009E,B06009_012E,B06008_005E,B06009_017E,B06009_001E,B06010_001E,B06008_002E,B01001H_001E,B06009_003E,B06009_018E,B06009_002E,B01001_026E,B06009_011E,ST,B00002_001E,B06009_008E,B06010_002E,B01001B_001E,B01001B_017E,B01001I_017E,B06009_006E,B06008_003E,B01001_001E,B01002_002E,B01002_003E,B01001A_002E,B06009_015E,B06008_004E,B01001_002E,COUNTY,B01001I_001E,B01002_001E,B06009_014E,B01001A_001E,B06009_004E,B01001B_002E,B01001A_017E,B05001_002E,B06009_005E,B01001I_002E,B01001H_002E,B06009_016E,NAME,B05001_005E,B06009_007E,B06009_010E&for=county:*&in=state:*
http://api.census.gov/data/2012/acs5?key=6c3fba83f5ccd2adb2fd3894045431e20e3ec5cb&get=B06012_018E,B06010_005E,B06012_002E,B06012_019E,B06012_020E,B06010_010E,B06011_003E,NAME,B06012_006E,B07004A_006E,B07001_017E,B06012_012E,COUNTY,B07004A_003E,B06010_009E,B06010_011E,B07001_049E,B06010_004E,B06012_

In [9]:
AllCounty = str(AllCounty)
splits = AllCounty.split(']')

#Making dict to store all values
fields = splits[0][5:].split('","')
fields1 = {}
for key in fields:
    fields1['%s' % key] = []

In [10]:
#parsing out line to add to fields1 dict, then convert to df
for line in splits[1:]:
    try:
        line = line.split('[')[1]
        line = line.split('null')
        if len(line) > 1: # skip entries with null
            pass
        elements = line[0].split('","')
        if (len(elements) == len(fields1.keys())):
            for field, element in zip(fields, elements):
                fields1['%s' % field].append(element.strip('"'))
    except:
        pass

In [11]:
AllCounty2 = str(AllCounty2)
splits = AllCounty2.split(']')

#Making dict to store all values
fields = splits[0][5:].split('","')
fields2 = {}
for key in fields:
    fields2['%s' % key] = []

In [12]:
#parsing out line to add to fields1 dict, then convert to df
for line in splits[1:]:
    try:
        line = line.split('[')[1]
        line = line.split('null')
        if len(line) > 1: # skip entries with null
            pass
        elements = line[0].split('","')
        if (len(elements) == len(fields2.keys())):
            for field, element in zip(fields, elements):
                fields2['%s' % field].append(element.strip('"'))
    except:
        pass

In [13]:
AllCounty3 = str(AllCounty3)
splits = AllCounty3.split(']')

#Making dict to store all values
fields = splits[0][5:].split('","')
fields3 = {}
for key in fields:
    fields3['%s' % key] = []

In [14]:
#parsing out line to add to fields1 dict, then convert to df
for line in splits[1:]:
    try:
        line = line.split('[')[1]
        line = line.split('null')
        if len(line) > 1: # skip entries with null
            pass
        elements = line[0].split('","')
        if (len(elements) == len(fields3.keys())):
            for field, element in zip(fields, elements):
                fields3['%s' % field].append(element.strip('"'))
    except:
        pass

Merging the Data frames

In [15]:
df = pd.DataFrame(fields1)
df2 = pd.DataFrame(fields2)
df3 = pd.DataFrame(fields3)

In [16]:
print(df.shape)
print(df2.shape)
print(df3.shape)

(3143, 52)
(2829, 43)
(3221, 40)


In [17]:
finalDf = pd.merge(pd.merge(df, df2, on='NAME'), df3, on='NAME', how='outer')

In [18]:
finalDf['fips_code'] = (finalDf['ST'] + finalDf['COUNTY']).astype('int')

In [19]:
finalDf = pd.merge(election, finalDf, on='fips_code')

In [20]:
for col in ['COUNTY', 'ST', 'county"', 'state', 'COUNTY_y', 'ST_y', 'county"_y','state_y']:
    del finalDf[col]

# DATAFRAME TO USE

In [21]:
#there might be some NAs in the demographics field, not sure what we want to do about them
#drop the county? what if it's an important county?
finalDf.head()

fips_code              county  total_2008  dem_2008  gop_2008  oth_2008  \
0      26041        Delta County       19064      9974      8763       327   
1      48295     Lipscomb County        1256       155      1093         8   
2       1127       Walker County       28652      7420     20722       510   
3      48389       Reeves County        3077      1606      1445        26   
4      56017  Hot Springs County        2546       619      1834        93   

   total_2012  dem_2012  gop_2012  oth_2012     ...      B19101_008E  \
0       18043      8330      9533       180     ...              581   
1        1168       119      1044         5     ...               30   
2       28497      6551     21633       313     ...             1309   
3        2867      1649      1185        33     ...              140   
4        2495       523      1894        78     ...               58   

   B19101_009E  B19101_010E  B19101_011E B19101_012E B19101_013E B19101_014E  \
0          586          709         1156        1394        1524         773   
1           31           23          127         102         141          75   
2          953          782         1471        2099        2122        1626   
3          113           33          187         229         468         188   
4           51           85          101         274         149         105   

  B19101_015E B19101_016E B19101_017E  
0         350         282         134  
1          53          18          29  
2         611         335         261  
3          29          90           8  
4          44          30          46  

[5 rows x 139 columns]

Try and predict change in number of voters over the years. 

Try and predict the change in demographic voting ratio. 

The project sheet said we have to use some clustering techniques to see which county was mis-clustered. Not sure if this is the best way to predict tho..

Also the sheet mentioned about building models that will explain things, so I was thinking of using Lasso or elastic net, and we can pull census data from 2011 and 2015 to try to see which variable were the most important features we from each data set